## 임상시험 데이터 확인
### 목적
임상시험 데이터 및 포맷, 스펙(데이터셋 스키마)를 확인하고 임상시험 데이터의 최종적 형태인 sas7bdat/xport를 다룬다.

### 준비
환경: 64비트 anaconda/conda\
임상시험 데이터: [cafe.naver.com/dmisimportant/104](cafe.naver.com/dmisimportant/104)

### Libraries

In [2]:
import os
import numpy as np
import pandas as pd
import secrets

### Variables
CRF 포맷, LAB 데이터 포맷, 데이터셋 파일 경로, 기타 변수 설정

In [3]:
gen=np.random.default_rng(2330)
ornament="-"*10
hangul="가나다라마바사아자차카파타하"
ext=(".sas7bdat",".xport")
path_crf="C:/code/CUBEDEMO2017/CUBEDEMO_2017_dc_or_fmt.xlsx"
path_lab="C:/code/CUBEDEMO2017/CUBEDEMO_2017_dc_or_lar.xlsx"
path_set="C:/code/CUBEDEMO2017/SASSET/"

### 포맷 파일 로드
CRF 포맷 파일에서 데이터 도메인 확인

In [4]:
form_crf=pd.read_excel(path_crf)
form_lab=pd.read_excel(path_lab)
print(ornament,"domain formats:",os.linesep,form_crf.DOMAIN.unique())

---------- domain formats: 
 ['AE' 'AY' 'CM' 'CT' 'CY' 'DM' 'DS' 'EF' 'EG' 'ES' 'IE' 'IP' 'LB' 'LC'
 'LY' 'MH' 'MY' 'PD' 'PG' 'RN' 'SU' 'SV' 'VS']


### 데이터셋 확인
데이터셋 경로의 sas7bdat / xport 파일을 os.path 오브젝트로 로드하고 불량 파일이 있는지 확인

In [5]:
sasobj=[obj for obj in os.scandir(path_set) if any(map(obj.path.lower().__contains__,ext)) and obj.is_file()]
sasbad=[obj for obj in sasobj if obj.stat().st_size<3]
if len(sasbad)>1:raise Exception("exotic file exists")

sas7bdat / xport os.path 오브젝트를 도메인 이름: 데이터프레임의 딕셔너리로 로드하고 어떤 도메인의 데이터셋이 있는지 확인
- 데이터셋 내 string이 bytes이므로 utf-8로 변환하고, 이 과정에서 누락되는 레코드가 있는지도 확인

In [6]:
def _decode(filepath):
    data=pd.read_sas(filepath)
    nas=data.notna().value_counts().sum()
    bytecol=data.select_dtypes("object").columns
    data[bytecol]=data[bytecol].apply(lambda q:q.str.decode("utf-8"))
    if nas==data.notna().value_counts().sum():
        return data
    else:
        print(ornament,"error:",filepath)
        return None

data={os.path.splitext(obj.name)[0].upper():_decode(obj.path) for obj in sasobj}
print(ornament,"domain:",os.linesep,data.keys(),os.linesep,len(data),"domains")

---------- domain: 
 dict_keys(['AE', 'AY', 'CM', 'CT', 'CY', 'DA', 'DM', 'DS', 'DY', 'EF', 'EG', 'EN', 'ES', 'IE', 'IP', 'LB', 'LC', 'LY', 'MH', 'MY', 'PD', 'PG', 'RN', 'SN', 'SU', 'SV', 'VS']) 
 27 domains


### 도메인별 데이터셋 형태 확인
SN은 PI 이름(서명)과 visit site ID를 포함
- SN.SNNAME은 subject 이름이 아니라 per-site PI 이름

In [7]:
sn=data["SN"]
sn.sample(10)

,SUBJID,VISIT,SNNAME,SNDTC
7,S-2Z-017,5007.0,황태중,2017-02-21
15,S-3Z-013,3006.0,김민지,2017-03-02
16,S-3Z-048,5007.0,송나경,2017-06-29
14,S-2Z-035,5007.0,공우석,2017-02-20
6,S-2Z-017,3006.0,황태중,2017-02-21
20,S-MJ-002,5007.0,김민지,2017-06-20
4,S-2Z-013,5007.0,신서란,2017-02-20
12,S-2Z-032,5007.0,송나경,2017-02-20
11,S-2Z-029,5007.0,공우석,2017-02-20
9,S-2Z-028,5007.0,공우석,2017-02-20


#### SN: SNNAME 가리기
PI 이름 가림

In [8]:
def _aname(name,n=2,chars=hangul):
    suffix="".join([secrets.choice(chars) for q in range(n)])
    return name[:n]+suffix

sn_snname_mapper={name:_aname(name) for name in sn.SNNAME.unique()}
sn.SNNAME.replace(sn_snname_mapper,inplace=True)
data["SN"].sample(10)

,SUBJID,VISIT,SNNAME,SNDTC
8,S-2Z-023,5007.0,송도하타,2017-02-20
6,S-2Z-017,3006.0,황태하카,2017-02-21
12,S-2Z-032,5007.0,송나타나,2017-02-20
19,S-MJ-001,5007.0,김민차카,2017-03-24
4,S-2Z-013,5007.0,신서사하,2017-02-20
23,S-MJ-010,5007.0,김민차카,2017-06-14
18,S-4Z-001,5007.0,원유라사,2017-03-07
2,S-2Z-006,5007.0,김민차카,2017-02-20
26,S-US-007,5007.0,이상자마,2017-03-27
24,S-US-002,5007.0,이상자마,2017-03-23


#### SN: SUBJID 추출
SN의 SUBJID를 각 도메인의 인덱스로 씀
- SN.SUBJID 개수를 PI 서명된 subject 수로 가정함

In [9]:
ix=sn.SUBJID.unique()
print(ix)
print(ornament,"total subjects:",len(ix))

['S-1Z-031' 'S-1Z-034' 'S-2Z-006' 'S-2Z-012' 'S-2Z-013' 'S-2Z-016'
 'S-2Z-017' 'S-2Z-023' 'S-2Z-028' 'S-2Z-029' 'S-2Z-032' 'S-2Z-034'
 'S-2Z-035' 'S-3Z-013' 'S-3Z-048' 'S-4Z-001' 'S-MJ-001' 'S-MJ-002'
 'S-MJ-009' 'S-MJ-010' 'S-US-002' 'S-US-003' 'S-US-007']
---------- total subjects: 23


#### DM: 내용 확인
DM은 인구학적 정보를 포함하며, 왜인지 인덱스(SN.SUBJID)에 부합하지 않는 row가 많음
- 인구학적 정보는 시험 초기에 수집되므로 인덱스에 해당하지 않는 row가 있으면 안됨
    - PI 서명이 안된 subjects

In [10]:
dm=data["DM"]
dm
dm[dm.SUBJID.isin(ix)]

,SUBJID,BRTHDTC,AGE,SEX,FERTILE
23,S-1Z-031,1984-09,32.0,2.0,1.0
26,S-1Z-034,1977-11,38.0,2.0,1.0
37,S-2Z-006,1975-04,41.0,2.0,2.0
43,S-2Z-012,1977-10,39.0,2.0,1.0
44,S-2Z-013,1991-07,25.0,1.0,NaN
47,S-2Z-016,1987-02,29.0,1.0,NaN
48,S-2Z-017,1983-03,33.0,1.0,NaN
54,S-2Z-023,1998-04,17.0,1.0,NaN
58,S-2Z-028,1989-04,27.0,2.0,2.0
59,S-2Z-029,1975-04,41.0,1.0,NaN


#### AE: 내용 확인
AE는 시험간 subject 별 adverse effect 정보를 포함
- 컬럼(e.g. form)별 숫자 코딩된 categorical data가 있음
- 상기에 따라 MedDRA 코딩된 컬럼이 있음
- 숫자 코딩의 기준은 CRF Format에 정의되어 있음

In [11]:
ae=data["AE"]
print(ae.loc[:,[col for col in ae if col.startswith("AE")]].sample(10))
print(ae[ae.SUBJID.isin(ix)][["SUBJID","INV_PT"]].sample(10))
form_crf[form_crf.DOMAIN=="AE"].sample(10)

                AETERM     AESTDTC  AETEAE  AEOUT     AEENDTC  AESER  AESEV  \
4               Asthma  2016-06-18     1.0    4.0  2016-06-22    3.0    2.0   
190       Headache NOS  2017-02-01     2.0    1.0  2017-02-03    3.0    2.0   
135      Stroke volume  2016-07-13     1.0    NaN         NaN    NaN    NaN   
144             Nausea  2016-06-22     1.0    5.0         NaN    1.0    1.0   
6      Angina pectoris  2016-06-15     1.0    NaN  2016-06-15    NaN    NaN   
25           Dizziness  2016-02-25     2.0    3.0  2016-02-26    1.0    1.0   
35   Anemia aggravated  2016-10-31     1.0    2.0         NaN    1.0    2.0   
183     Liver ablation  2016-04-29     2.0    3.0  2016-05-UK    1.0    1.0   
31               Fever  2016-10-26     2.0    2.0         NaN    1.0    1.0   
89    Lingual dystonia  2017-02-04     1.0    2.0         NaN    3.0    2.0   

     AEREL  AEACN  AEACNOTH  
4      3.0    3.0       2.0  
190    2.0    1.0       2.0  
135    NaN    NaN       NaN  
144    2.0

,DOMAIN,FMTNAME,VARNAME,START,END,LABEL
10,AE,AEACNOTH,AEACNOTH,4,4,Drug and non-drug treatment
23,AE,AESER,AESER,3,3,Hospitalization
12,AE,AEOUT,AEOUT,2,2,Not recovered/Not resolved
17,AE,AEREL,AEREL,1,1,Not related
1,AE,AEACN,AEACN,2,2,Dose not changed
29,AE,AESEV,AESEV,2,2,Moderate
18,AE,AEREL,AEREL,2,2,Unlikely related
27,AE,AESER,AESER,7,7,Other medically important event
25,AE,AESER,AESER,5,5,Congenital anomaly or birth defect
19,AE,AEREL,AEREL,3,3,Possibly related


#### AE: AE Code Labeling
AE의 코딩된 categorical AEs이 CRF 상에서 무엇을 의미하는지 확인하고자 함
- DB Specification 또는 CRF Format에서 컬럼별 코드 및 라벨을 컬럼: 코드: 라벨의 딕셔너리로 추출
- 딕셔너리에 따라 컬럼 코드를 라벨링

In [12]:
def _get_label(form,var="VARNAME",kvp=["END","LABEL"]):
    label={}
    for varname in form[var].unique():
        kv=form[form[var]==varname].loc[:,kvp].values
        label[varname]={k:v for k,v in kv}
    return label

ae_label=_get_label(form_crf)
print(ae_label)

{'AEACN': {1: 'Dose increased', 2: 'Dose not changed', 3: 'Dose reduced', 4: 'Drug interrupted', 5: 'Drug withdrawn', 6: 'Not applicable', 7: 'Unknown'}, 'AEACNOTH': {1: 'None', 2: 'Drug treatment', 3: 'Non-drug treatment', 4: 'Drug and non-drug treatment'}, 'AEOUT': {1: 'Fatal', 2: 'Not recovered/Not resolved', 3: 'Recovered/Resolved', 4: 'Recovered/Resolved with sequelae', 5: 'Recovering/Resolving', 6: 'Unknown'}, 'AEREL': {1: 'Not related', 2: 'Unlikely related', 3: 'Possibly related', 4: 'Related'}, 'AESER': {1: 'No', 2: 'Death', 3: 'Hospitalization', 4: 'Life threatening', 5: 'Congenital anomaly or birth defect', 6: 'Significant disability', 7: 'Other medically important event'}, 'AESEV': {1: 'Mild', 2: 'Moderate', 3: 'Severe'}, 'AETEAE': {1: 'Yes', 2: 'No'}, 'AEYN': {1: 'Yes', 2: 'No'}, 'CMINDC': {1: 'Medical history', 2: 'Adverse event', 3: 'Hypertension', 4: 'Prophylaxis', 5: 'Supplement', 6: 'Other'}, 'CMONGO': {1: 'Checked'}, 'CTYN': {1: 'Yes', 2: 'No'}, 'CMYN': {1: 'Yes', 2:

In [13]:
ae_col_ambi=[col for col in ae if col in form_crf[form_crf.DOMAIN=="AE"].VARNAME.values]
ae[ae_col_ambi].replace(ae_label)

,AETEAE,AEOUT,AESER,AESEV,AEREL,AEACN,AEACNOTH
0,Yes,Not recovered/Not resolved,Life threatening,Severe,Unlikely related,Dose reduced,Non-drug treatment
1,Yes,Unknown,Death,Mild,Related,Drug interrupted,None
2,NaN,Recovering/Resolving,NaN,NaN,NaN,NaN,NaN
3,Yes,Fatal,Life threatening,Moderate,Unlikely related,Dose not changed,Drug treatment
4,Yes,Recovered/Resolved with sequelae,Hospitalization,Moderate,Possibly related,Dose reduced,Drug treatment
...,...,...,...,...,...,...,...
186,No,Unknown,No,Mild,Not related,Dose not changed,Drug treatment
187,Yes,Not recovered/Not resolved,Hospitalization,Mild,Unlikely related,Drug withdrawn,Drug treatment
188,Yes,Fatal,No,Mild,Unlikely related,Unknown,None
189,No,Not recovered/Not resolved,Other medically important event,Mild,Possibly related,Drug withdrawn,Drug and non-drug treatment


#### MH: Specification
MH 데이터의 스키마 불러오기
- 위와는 다르게 DB Specification을 사용
    - DB Specification에 데이터셋 스키마, 프론트앤드 디자인 등 CRF의 모든 것이 정의되어 있음

In [14]:
path_spec="C:/code/CUBEDEMO2017/spec.xlsx"
spec=pd.read_excel(path_spec).loc[:,:"VIEW_TYPE"]
spec.CODE.sample(5)

244           NaN
127    1:Yes|2:No
62            NaN
40            NaN
169           1:1
Name: CODE, dtype: object

- DB Spec-sheet의 CODE 컬럼 내용에 NaN과 문자열이 혼재함
- 차후 편한 앤코딩 및 디코딩을 위해 코딩 규칙이 있는 경우 (notna) 딕셔너리로 바꿈

In [15]:
spec["CODE"]=spec.CODE.apply(lambda q:dict(item.split(":") for item in q.split("|")) if pd.notna(q) else q).dropna()
spec.CODE.sample(5)

100    NaN
85     NaN
245    NaN
271    NaN
9      NaN
Name: CODE, dtype: object

- MH 데이터셋 불러오기 및 주요 컬럼 확인

In [16]:
mh=data["MH"]
mh[[col for col in mh if col.startswith("MH")]].sample(5)

,MHTERM,MHONGO,MHENDTC,MHCONTRT,MHCO
82,Spinal pain,2.0,2016-11-UK,1.0,NaN
78,Deaf,1.0,NaN,2.0,none
125,Nausea,2.0,2017-03-01,1.0,disease finished
88,Dialysis,1.0,NaN,1.0,Taken about 3 months ago.
23,Anaemia,2.0,2016-08-20,1.0,NaN


- Spec-sheet에서 데이터 컬럼 스키마를 냄

In [17]:
def _label(series,spec):
    spec=spec[spec.ITEMID==series.name]
    crfname=spec.CRF_LABEL.iat[0]
    name=spec.ITEM_LABEL.iat[0]
    code=spec.CODE.iat[0]
    visit=spec.VISIT.iat[0]
    return crfname,name,code,visit

mh_MHONGO_spec=_label(mh.MHONGO,spec)
print(ornament,"\nCRF Name: ",mh_MHONGO_spec[0],"\nForm Label: ",mh_MHONGO_spec[1],"\nCode: ",mh_MHONGO_spec[2])

---------- 
CRF Name:  Medical History 
Form Label:  Ongoing 
Code:  {'1': 'Yes', '2': 'No'}


- MH의 MHONGO는 CRF의 Medical History 부문에 있고, ongoing 여부를 묻는 binary 질문임

#### LB: 내용 확인
LB는 랩 테스트 결과를 포함
- VISIT, SEQ 등 랩 테스트 회차에 따른 롱 포맷으로 되어 있어 인덱스를 정할 때 유의해야 함
- LB.LBTEST에 해당 랩 테스트 명목이 나와 있음
- LB.LBORRES에 랩 결과 수치가 나와 있음
    - 일부 categorical data 포함 (e.g. Occult Blood)

In [18]:
lb=data["LB"]
lb.sample(10)

,SUBJID,VISIT,SEQ,LBTEST,LBORRES,LBNOR,LBCLSIG
3349,S-MJ-010,3.0,13.0,"Glomerular Filtration Rate, Estimated [Automat...",101.81,NaN,NaN
609,S-1Z-032,3.0,12.0,Creatinine,1.77,1.0,NaN
1080,S-2Z-011,2.0,13.0,"Glomerular Filtration Rate, Estimated [Automat...",153.97,NaN,NaN
1485,S-2Z-022,4.0,4.0,Platelets,152.00,1.0,NaN
625,S-1Z-033,1.0,10.0,Bilirubin,1.50,1.0,NaN
180,S-1Z-020,1.0,16.0,pH[U],7.00,1.0,NaN
560,S-1Z-031,1.0,18.0,Occult blood[U],NaN,1.0,NaN
3543,S-US-005,1.0,2.0,Hemoglobin,1.00,NaN,NaN
1143,S-2Z-013,1.0,4.0,Platelets,390.00,1.0,NaN
17,S-1Z-010,1.0,9.0,Alanine Aminotransferase,150.00,NaN,NaN


In [19]:
lb_desc=lb.groupby(["LBTEST"])["LBORRES"].agg(["mean","std"])
lb_desc

,mean,std
LBTEST,,
Alanine Aminotransferase,336.257126,1409.494132
Albumin,335.514705,1577.394172
Albumin [U],NaN,NaN
Aspartate Aminotransferase,325.924723,1415.747206
Bilirubin,215.007593,1165.893477
Creatinine,184.471609,1056.242446
Erythrocytes,245.251260,1130.807728
"Glomerular Filtration Rate, Estimated [Automatical",5003.627111,3658.624184
"Glomerular Filtration Rate, Estimated [Automatically calculated]",82.635866,40.426010


#### LB: Mockup Data 만들기
Subject Index와 LB를 이용해서 목업 데이터를 만듦
- SN.SUBJID를 기준으로 피험자당 10번의 랩 테스트를 가정
    - 정규분포로 가정
    - Albuminuria 등 bad result 여부는 발생률 10%로 가정
- 편의상 DM을 결합

In [23]:
def _gen_mockup(desc,count):
    print(ornament,"generating values")
    return {q:gen.normal(desc.loc[q]["mean"],desc.loc[q]["std"],count) if pd.notna(desc.loc[q]["std"])
             else gen.binomial(1,.1,count) for q in desc.index}

def gen_mockup(desc,ix,count=10):
    data=_gen_mockup(desc,count=len(ix)*count)
    ix=pd.MultiIndex.from_product([ix,[q for q in range(1,count+1,1)]],names=["SUBJID","VISIT"])
    return pd.DataFrame(data,ix).reset_index()

mockup=gen_mockup(lb_desc,ix).merge(dm,on="SUBJID")
mockup

---------- generating values


,SUBJID,VISIT,Alanine Aminotransferase,Albumin,Albumin [U],Aspartate Aminotransferase,Bilirubin,Creatinine,Erythrocytes,"Glomerular Filtration Rate, Estimated [Automatical",...,Occult blood[U],Platelets,Protein,Protein[U],Specific Gravity[U],pH[U],BRTHDTC,AGE,SEX,FERTILE
0,S-1Z-031,1,196.749342,2623.839327,0,1256.383186,2120.090254,-260.278179,17.943961,9828.841516,...,0,2299.273869,3478.826118,0,445.439622,3170.924321,1984-09,32.0,2.0,1.0
1,S-1Z-031,2,2520.461693,265.292345,1,2318.746475,399.674536,-417.745876,-445.803171,9645.199675,...,0,1113.360519,-1431.976410,0,-1588.108772,-1308.676970,1984-09,32.0,2.0,1.0
2,S-1Z-031,3,329.639062,2112.128194,0,-988.821948,-595.320376,-983.050569,-72.343967,-2151.714218,...,1,-1742.625409,-668.159473,0,779.187792,851.319071,1984-09,32.0,2.0,1.0
3,S-1Z-031,4,-1681.291488,849.607638,0,1009.326435,-329.635315,1015.822042,490.732474,5786.864707,...,0,2131.021103,-916.022305,0,-989.746623,1568.509999,1984-09,32.0,2.0,1.0
4,S-1Z-031,5,134.744336,2385.238673,0,-2583.667456,1021.313373,236.695699,-628.025457,13911.826792,...,0,837.014933,-2361.431060,0,-508.989057,5.793759,1984-09,32.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,S-US-007,6,257.999632,3206.301449,0,3020.891786,1546.617101,-935.729952,64.455093,595.910931,...,0,2900.991463,625.242359,0,416.947802,-244.209072,1981-01,36.0,1.0,NaN
226,S-US-007,7,1400.043218,2134.026706,0,561.560217,-592.399261,273.676705,-161.926375,3762.120004,...,1,-2565.057917,1762.138551,0,-1760.283932,-325.112188,1981-01,36.0,1.0,NaN
227,S-US-007,8,225.665225,-699.912463,0,-439.872189,-811.638376,814.309076,584.368175,6900.305477,...,0,2160.567065,939.341683,0,110.614381,-385.001606,1981-01,36.0,1.0,NaN
228,S-US-007,9,757.513609,-1227.072995,0,593.037057,1166.860585,-244.950011,-89.478455,3057.370903,...,0,1721.254980,-1587.643160,0,-94.892306,529.709767,1981-01,36.0,1.0,NaN


- Categorical value가 제대로 생성됐는지 확인

In [24]:
mockup[lb_desc[pd.isna(lb_desc["std"])].index]

,Albumin [U],Occult blood[U],Protein[U]
0,0,0,0
1,1,0,0
2,0,1,0
3,0,0,0
4,0,0,0
...,...,...,...
225,0,0,0
226,0,1,0
227,0,0,0
228,0,0,0


### 후기
- 하나의 임상시험을 구성하는 데이터셋은 많으며, 각 데이터 연동을 위한 정의와 연동이 일목요연히 되어 있음
    - 수치형, 범주형 값이 혼재함
        - 범주형 데이터인데 숫자로 코딩되고 수치형 값과 기술적으로 구분되지 않는 것을 유의해야 함
    - 일반적이지 않은 범주형 값 (0,1(No,Yes(False,True))가 아닌 1,2(Yes,No))
- 프로토콜 이해를 선행하지 않으면 데이터셋을 정확히 다루기 어려움
    - Visit Number / Window, Endpoint, Inclusion / Exclusion Criteria
    - Longitudinal Study 여부 (endpoint가 time series인지)
- Specsheet는 엔드유저에게 해당 데이터가 어떻게 보이고 수집됐는지를 포함
- 도메인 데이터와 DB Specification 내 각 도메인 정의를 연동할 수 있는 클래스 이용 필요